Based on @yaov notebook "The unreasonable effectiveness of Character-level Language Models" (http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139).

Using the ** unsmoothed maximum-liklihood character level language models ** to generate text passing first words, which can be seen as defining the subject.

In [1]:
from collections import *
import random

def train_char_lm(fname, order=4):
    data = file(fname).read()
    lm = defaultdict(Counter)
    pad = "~" * order
    data = pad + data
    for i in xrange(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.iteritems()]
    outlm = {hist:normalize(chars) for hist, chars in lm.iteritems()}
    return outlm

def generate_letter(lm, history, order):
        history = history[-order:]   

        if len(history) < order:
            possible_hist = [key for key, value in lm.items() if history.lower() == key.lower()[-len(history):]]
            history = possible_hist[random.randint(0, len(possible_hist)-1)]
      
        try:
            dist = lm[history]
        except:
            dist = lm[history.lower()]
        x = random.random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c

def generate_text(lm, order, nletters=1000, history=None):
    if history == None: 
        history = "~" * order
    else:
        history = history[:min(len(history), order)]
    out = []
    out.append(history)
    while(len(out) < nletters or out[-1] != "."):
    #for i in xrange(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

In [ ]:
lm = train_char_lm("data/proust_dataset_ptbr.txt", order=18)

In [ ]:
print generate_text(lm, 20, nletters=300, history="Seios")